In [138]:
import pandas
import pickle
import sklearn
from sklearn.neural_network import MLPClassifier
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from sklearn import preprocessing


%matplotlib inline
np.random.seed(1)

# Import pandas df

In [6]:
# import the data
data = pandas.read_pickle('raw_processed.pkl')
dataShort = data.iloc[0:10,19:46].copy()
data

,TIMESTAMP,LINE_NO,UTMX,UTMY,ELEVATION,CHANNEL_NO,RX_ALTITUDE,RX_ALTITUDE_STD,TX_ALTITUDE,TX_ALTITUDE_STD,...,DBDT_INUSE_Ch2GT31,DBDT_INUSE_Ch2GT32,DBDT_INUSE_Ch2GT33,DBDT_INUSE_Ch2GT34,DBDT_INUSE_Ch2GT35,DBDT_INUSE_Ch2GT36,DBDT_INUSE_Ch2GT37,closest_pos,closest_ind,VALID
0,43434.663700,100101,573558.0,4395767.7,63.11,1,49.11,0.060,50.55,0.060,...,9999,9999,9999,9999,9999,9999,9999,12.776932,11198.0,True
1,43434.663712,100101,573558.5,4395758.1,63.15,2,48.99,0.060,50.82,0.060,...,1,1,1,1,1,0,1,3.228002,11198.0,True
2,43434.663715,100101,573559.3,4395755.2,63.16,1,48.89,0.060,50.76,0.060,...,9999,9999,9999,9999,9999,9999,9999,0.223607,11198.0,True
3,43434.663727,100101,573564.0,4395745.9,63.16,2,47.32,0.062,49.20,0.062,...,1,1,1,1,1,0,1,0.141421,11200.0,True
4,43434.663731,100101,573566.0,4395743.1,63.15,1,46.60,0.063,48.41,0.063,...,9999,9999,9999,9999,9999,9999,9999,3.580503,11200.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7316,43436.748160,710101,579762.1,4390268.5,45.84,1,61.36,0.050,57.98,0.050,...,9999,9999,9999,9999,9999,9999,9999,344.506734,11709.0,False
7317,43436.748172,710101,579763.7,4390244.7,45.84,2,54.92,0.056,51.78,0.056,...,0,0,0,0,0,0,1,361.668536,11709.0,False
7318,43436.748176,710101,579764.3,4390237.4,45.84,1,53.01,0.058,50.01,0.058,...,9999,9999,9999,9999,9999,9999,9999,366.785932,11711.0,False
7319,43436.748188,710101,579766.2,4390212.7,45.84,2,47.11,0.065,44.69,0.065,...,0,0,0,0,0,0,1,384.290957,11711.0,False


# Getting correct data shpe and format using code from Noah's notebook

In [10]:
lineNumbers=np.unique(data["LINE_NO"])
timeDiff=np.diff(data["TIMESTAMP"])
timeDiffMask=timeDiff<5e-6
# set the number of adjacent soundings to use
adSoundings=1;

# iterate through the line numbers
for line in lineNumbers:

    # make a mask for the rows in the big DF that are for this line number
    rowIndex=data["LINE_NO"]==line
    
    # get just the current rows
    currData=data.loc[rowIndex,:]
    currData.reset_index(drop=True, inplace=True)
    
    # make an array of the time differences
    timeDiff=np.diff(currData["TIMESTAMP"])

    # if the first time difference is large, drop the first row, this throws a warning
    if timeDiff[0]>1e-5:
        currData.drop([0], inplace=True)
        currData.reset_index(drop=True, inplace=True)
    
    # set the number of pairs in this line and make some arrays with zeros for the HM and LM data
    numberPairs=np.int(len(currData.index)/2)
    currDataLM=np.zeros([numberPairs,1+adSoundings*2,37])
    currDataHM=np.zeros([numberPairs,1+adSoundings*2,37])
    currLabelsLM=np.zeros([numberPairs,1])
    currLabelsHM=np.zeros([numberPairs,1])
    # check to see if the first row is LM or HM, set the indexing
    if np.mean(currData.loc[0,'DBDT_Ch1GT1':'DBDT_Ch1GT28'])>9990:
        hmFirst=1
    else:
        hmFirst=0
        
        
    # iterate through the number of pairs
    for i in range(numberPairs):
        if hmFirst==1:
            if adSoundings==1:
                hmIndex=[(i-1)*2,i*2,(i+1)*2]
                lmIndex=[(i-1)*2+1,i*2+1,(i+1)*2+1]
            elif adSoundings==2:
                hmIndex=[(i-2)*2,(i-1)*2,i*2,(i+1)*2,(i+2)*2]
                lmIndex=[(i-2)*2+1,(i-1)*2+1,i*2+1,(i+1)*2+1,(i+2)*2+1]
        else:
            if adSoundings==1:
                lmIndex=[(i-1)*2,i*2,(i+1)*2]
                hmIndex=[(i-1)*2+1,i*2+1,(i+1)*2+1]
            elif adSoundings==2:
                hmIndex=[(i-2)*2,(i-1)*2,i*2,(i+1)*2,(i+2)*2]
                lmIndex=[(i-2)*2+1,(i-1)*2+1,i*2+1,(i+1)*2+1,(i+2)*2+1]
        
        # do something different for the first and last sounding, leave the adjacent sounding as 0s
        if i==0:
            currDataLM[i,2,0:28]=currData.loc[lmIndex[2],'DBDT_Ch1GT1':'DBDT_Ch1GT28']
            currDataHM[i,2,:]=currData.loc[hmIndex[2],'DBDT_Ch2GT1':'DBDT_Ch2GT37']
        elif i==numberPairs-1:
            currDataLM[i,0,0:28]=currData.loc[lmIndex[0],'DBDT_Ch1GT1':'DBDT_Ch1GT28']
            currDataHM[i,0,:]=currData.loc[hmIndex[0],'DBDT_Ch2GT1':'DBDT_Ch2GT37']
            
        # for the rest fill in both adjacent soundings
        else:
            currDataLM[i,0,0:28]=currData.loc[lmIndex[0],'DBDT_Ch1GT1':'DBDT_Ch1GT28']
            currDataLM[i,2,0:28]=currData.loc[lmIndex[2],'DBDT_Ch1GT1':'DBDT_Ch1GT28']
            currDataHM[i,0,:]=currData.loc[hmIndex[0],'DBDT_Ch2GT1':'DBDT_Ch2GT37']
            currDataHM[i,2,:]=currData.loc[hmIndex[2],'DBDT_Ch2GT1':'DBDT_Ch2GT37']
            
        # middle sounding and labels always get set
        currDataLM[i,1,0:28]=currData.loc[lmIndex[1],'DBDT_Ch1GT1':'DBDT_Ch1GT28']
        currDataHM[i,1,:]=currData.loc[hmIndex[1],'DBDT_Ch2GT1':'DBDT_Ch2GT37']
        currLabelsLM[i,0]=currData.loc[i*2+1,'VALID']
        currLabelsHM[i,0]=currData.loc[i*2,'VALID']
    
    if line==lineNumbers[0]:
        dataLM=currDataLM
        dataHM=currDataHM
        labelsLM=currLabelsLM
        labelsHM=currLabelsHM
    else:
        dataLM=np.append(dataLM,currDataLM, axis=0)
        dataHM=np.append(dataHM,currDataHM, axis=0)
        labelsLM=np.append(labelsLM,currLabelsLM, axis=0)
        labelsHM=np.append(labelsHM,currLabelsHM, axis=0)

# drop rows where the lm and hm labels dont agree
indexGood=labelsHM==labelsLM

# see how many locations this happened in
print(np.sum(np.abs(labelsLM-labelsHM)))

# keep only the rows where they agree
dataLM=dataLM[indexGood[:,0],:,:]
dataHM=dataHM[indexGood[:,0],:,:]
labelsLM=labelsLM[indexGood[:,0],:]
labelsHM=labelsHM[indexGood[:,0],:]

/Users/ankushs/anaconda3/envs/peeved/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


235.0


In [175]:
# make X from the HM and LM data, X is m (examples) by 3 (first adjacent sounding, middle sounding, other adjacent sounding) 
# by 37 (timegates) by 2 (low moment then high moment)
X=np.zeros([np.shape(dataLM)[0],np.shape(dataLM)[1],np.shape(dataLM)[2],2])
X[:,:,:,0]=dataLM
X[:,:,:,1]=dataHM

# set nans to 0 (there shouldnt be any but who knows)
X=np.nan_to_num(X)

# scale each timegate to be between -1 and 1
min_max_scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
for i in range(np.shape(X)[-1]):
    for j in range(np.shape(X)[1]):
        
        # do it for all examples, for this moment
        X[:,j,:,i] = min_max_scaler.fit_transform(X[:,j,:,i])
        
        # for the timegates where the LM never exists, set it back to 0 so it sits in the middle of the scaled range
        for k in range(np.shape(X)[2]):
            if i==0:
                if k>=28:
                    X[:,j,k,i]=X[:,j,k,i]-X[:,j,k,i]

# make some random indices, then take 20000 examples for training and the rest for test, results in about 85/15 split
indices=np.random.permutation(X.shape[0])
X_train=X[indices[:20000],:,:,:]
X_test=X[indices[20000:],:,:,:]
Y_train=labelsLM[indices[:20000],:]
Y_test=labelsLM[indices[20000:],:]

# print out some shit
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 20000
number of test examples = 3760
X_train shape: (20000, 3, 37, 2)
Y_train shape: (20000, 1)
X_test shape: (3760, 3, 37, 2)
Y_test shape: (3760, 1)


In [176]:
X_train.shape

(20000, 3, 37, 2)

# Reshaping data for FC layers only

In [177]:
X_train = np.reshape(X_train, (len(X_train), 3*37*2))
X_test = np.reshape(X_test,(len(X_test), 3*37*2))

# Simple NN using SKLEARN

In [226]:
#Simple NN using SKLEARN
clf = MLPClassifier(activation = 'relu', solver='adam', alpha=0.00001,hidden_layer_sizes=(10,5,5), random_state=1, tol = 1e-9, max_iter=10000)
clf.fit(X_train, np.ravel(Y_train))

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(10, 5, 5), learning_rate='constant',
              learning_rate_init=0.001, max_iter=10000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=1e-09,
              validation_fraction=0.1, verbose=False, warm_start=False)

# Simple NN using TF (FC Layers only)

In [228]:
test_accuracy = clf.score(X_test, Y_test)
print("The Test accuracy is " + str(test_accuracy))

The Test accuracy is 0.7627659574468085
